# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235142820955                   -0.50    9.0         
  2   -7.250462239317       -1.81       -1.41    1.0   8.60ms
  3   -7.251225826967       -3.12       -2.27    1.0   9.02ms
  4   -7.251326019738       -4.00       -2.45    2.0   10.9ms
  5   -7.251334967855       -5.05       -2.82    1.0   9.38ms
  6   -7.251338097392       -5.50       -3.14    2.0   10.7ms
  7   -7.251338658450       -6.25       -3.51    1.0   9.22ms
  8   -7.251338776463       -6.93       -3.79    1.0   9.65ms
  9   -7.251338795711       -7.72       -4.24    1.0   9.47ms
 10   -7.251338798353       -8.58       -4.75    2.0   10.9ms
 11   -7.251338798670       -9.50       -5.39    2.0   11.4ms
 12   -7.251338798702      -10.49       -5.84    3.0   12.0ms
 13   -7.251338798704      -11.67       -6.35    1.0   9.44ms
 14   -7.251338798704      -12.62       -6.52    3.0   12.4ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04899439555179215
[ Info: Arnoldi iteration step 2: normres = 0.37903979087514866
[ Info: Arnoldi iteration step 3: normres = 0.7142359935606966
[ Info: Arnoldi iteration step 4: normres = 0.5157018984544791
[ Info: Arnoldi iteration step 5: normres = 0.6213796391360442
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.60e-01, 8.22e-02, 5.44e-01, 2.40e-01, 2.30e-02)
[ Info: Arnoldi iteration step 6: normres = 0.22694711546222834
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.13e-02, 1.38e-01, 1.30e-01, 9.64e-02, 5.03e-02)
[ Info: Arnoldi iteration step 7: normres = 0.12952985975558223
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.93e-03, 1.57e-02, 1.36e-02, 8.76e-02, 8.21e-02)
[ Info: Arnoldi iteration step 8: normres = 0.0975952518612646
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (8.63e-